# Stock Analysis Notebook

## Table of Contents

- [Setup and Configuration](#setup-and-configuration)
- [Stock Selection](#stock-selection)  
- [Technical Analysis Dashboard](#technical-analysis-dashboard)
- [Company Information](#company-information)
- [Analyst Price Targets](#analyst-price-targets)
- [Market Intelligence](#market-intelligence)

[Back to Top](#table-of-contents)

## Setup and Configuration

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Core imports
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import scipy.stats as stats
from typing import Dict, List, Optional
from pprint import pprint
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import json
from io import BytesIO
import base64

# Add src to path and import custom functions
import sys
sys.path.insert(0, '../src')

from IPython.display import Markdown, display, HTML
from functions import (
    get_ticker, 
    fetch_stock_data, 
    setup_date_formatting, 
    humanize_number, 
    safe_float, 
    get_current_price_and_date,
    display_structured_data_as_markdown,
    plot_analyst_recommendations,
    display_balance_sheet_html,
    display_stock_metrics,
    calculate_capm_metrics,
    display_capm_analysis,
    plot_capm_analysis,
    format_holdings_markdown,
    display_company_holdings,
    get_daily_changes,
    create_world_map,
    save_world_map
)

In [ ]:
# Matplotlib Style Configuration
# Change this line to use different graph styles

plt.style.use('default')            # Clean white background

# Custom style settings (optional)
plt.rcParams.update({
    'figure.figsize': (12, 8),      # Default figure size
    'font.size': 10,                # Default font size
    'axes.labelsize': 12,           # Axis label size
    'axes.titlesize': 14,           # Title size
    'xtick.labelsize': 10,          # X-tick label size
    'ytick.labelsize': 10,          # Y-tick label size
    'legend.fontsize': 10,          # Legend font size
    'figure.dpi': 200,              # Resolution
})

# List available styles
#for i, style in enumerate(sorted(plt.style.available), 1):
    #print(f"  {i:2d}. {style}")

In [ ]:
# Utility function for making API requests to CNN Business
def make_cnn_api_request(url, max_retries=3, backoff_factor=1.0, timeout=10):
    """
    Make an API request to CNN Business with retry logic.
    
    Args:
        url: The API endpoint URL
        max_retries: Number of retry attempts
        backoff_factor: Factor for exponential backoff
        timeout: Request timeout in seconds
    
    Returns:
        Response object or None if all retries failed
    """
    session = requests.Session()
    retry = Retry(
        total=max_retries,
        backoff_factor=backoff_factor,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    try:
        response = session.get(url, timeout=timeout)
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

## Stock Selection

**How to use the stock selector:**
1. Type a stock symbol in the text box or select from the dropdown
2. Choose your desired time period
3. The dashboard will automatically update with the new stock data

In [ ]:
# Stock and Period Selection
# Run this cell ONCE to create the selectors, then use them to pick your stock

import ipywidgets as widgets
from IPython.display import display

# Popular stocks for quick selection
popular_stocks = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'NVDA', 'META', 'TSM', 
                  'JPM', 'V', 'WMT', 'JNJ', 'PG', 'MA', 'HD', 'DIS', 'NFLX', 'PYPL']

# Check if widgets already exist, if not create them
if 'stock_dropdown' not in globals():
    stock_dropdown = widgets.Dropdown(
        options=sorted(popular_stocks),
        value='AAPL',
        description='Stock:',
        style={'description_width': '80px'}
    )

if 'stock_text' not in globals():
    stock_text = widgets.Text(
        value='AAPL',
        placeholder='Enter ticker symbol',
        description='Stock Symbol:',
        style={'description_width': '100px'}
    )

if 'period_dropdown' not in globals():
    period_dropdown = widgets.Dropdown(
        options=['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'max'],
        value='1y',
        description='Period:',
        style={'description_width': '80px'}
    )

# Callback functions to update selected_stock and period
def update_from_dropdown(change):
    global selected_stock, period
    selected_stock = stock_dropdown.value
    print(f"Stock set to: {selected_stock}")

def update_from_text(change):
    global selected_stock
    text_value = stock_text.value.strip().upper()
    if text_value:
        selected_stock = text_value
        print(f"Stock set to: {selected_stock}")

def update_period(change):
    global period
    period = period_dropdown.value
    print(f"Period set to: {period}")

# Attach callbacks
stock_dropdown.observe(update_from_dropdown, names='value')
stock_text.observe(update_from_text, names='value')
period_dropdown.observe(update_period, names='value')

# Initialize variables
selected_stock = stock_dropdown.value
period = period_dropdown.value

print("Select your stock:")
display(stock_dropdown)
print("\nOr type any ticker:")
display(stock_text)
print("\nSelect time period:")
display(period_dropdown)
print("\nAfter selecting, run the next cell OR click 'Run All Below' to analyze")

In [ ]:
# Confirm selection
print(f"✓ Stock selected: {selected_stock}")
print(f"✓ Period: {period}")
print("\nRun the next cell to fetch data and analyze")

In [ ]:
# STOCK SELECTION
# Change the stock symbol below to any stock you want to analyze
print(f"Fetching data for {selected_stock}...")

# Fetch stock data
data = fetch_stock_data(selected_stock, period)

if not data.empty:
    # Calculate technical indicators using utility function
    from functions import calculate_technical_indicators
    data = calculate_technical_indicators(data)
    
    # Calculate additional metrics
    data['Daily_Return'] = data['Close'].pct_change()
    data['Volatility'] = data['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
    data['Cumulative_Return'] = (1 + data['Daily_Return']).cumprod() - 1
    
    ticker = get_ticker(selected_stock)  # Ensure ticker is set
    print(f"Successfully loaded {selected_stock}")
else:
    print(f"No data found for {selected_stock}")

## Technical Analysis Dashboard

In [ ]:
# Create dashboard visualization
print(f"Creating dashboard for {selected_stock}...")
if not isinstance(data,pd.DataFrame)or data.empty:print("Error: No valid stock data. Run STOCK SELECTION cell first.");raise SystemExit
pn={'1d':'1 Day','5d':'5 Days','1mo':'1 Month','3mo':'3 Months','6mo':'6 Months','1y':'1 Year','2y':'2 Years','5y':'5 Years','10y':'10 Years','ytd':'Year to Date','max':'Maximum'}
prd=pn.get(period,period);cp=data['Close'];ix,li=data.index,data.index[-1];rs20=cp.rolling(20).std();dr=data['Daily_Return']
ub,lb=data['MA_20']+(rs20*2),data['MA_20']-(rs20*2);e12,e26=cp.ewm(span=12).mean(),cp.ewm(span=26).mean();macd=e12-e26;sig=macd.ewm(span=9).mean();hist=macd-sig
cum=(1+dr).cumprod();dd=((cum-cum.cummax())/cum.cummax())*100;lhr=data['High'].rolling(14).max()-data['Low'].rolling(14).min()
kp=100*((cp-data['Low'].rolling(14).min())/lhr);dp=kp.rolling(3).mean();obv=(np.sign(cp.diff())*data['Volume']).fillna(0).cumsum();rsh=(dr.rolling(60).mean()/dr.rolling(60).std())*np.sqrt(252)

def an(a,v,x,c='#114f7a'):
    if not np.isnan(v):a.annotate(f'{v:.1f}',xy=(x,v),xytext=(5,5),textcoords='offset points',fontsize=8,fontweight='bold',bbox=dict(boxstyle='round,pad=0.2',facecolor=c,alpha=0.2))

def st(a,t,y='',l=True,tk=8):a.set_title(t,fontweight='bold',fontsize=11);a.set_ylabel(y,fontsize=9)if y else None;a.tick_params(axis='both',labelsize=tk);a.legend(fontsize=7)if l else None

fig=plt.figure(figsize=(14,16));gs=GridSpec(6,3,height_ratios=[4,1,1.5,1.5,1.5,2],hspace=0.4,wspace=0.3)
am=fig.add_subplot(gs[0,:]);am.plot(ix,cp,color="#114f7a",label=f'{selected_stock} Close',linewidth=2.5)
am.plot(ix,data['MA_20'],color="#114e7ae0",alpha=0.8,linestyle='--',label='20-day MA',linewidth=1.5)
am.plot(ix,data['MA_50'],color="#114e7ac8",alpha=0.8,linestyle=':',label='50-day MA',linewidth=1.5)
am.fill_between(ix,lb,ub,color='gray',alpha=0.2,label='Bollinger Bands')
am.set_title(f'{selected_stock} Stock Price with Moving Averages & Bollinger Bands ({prd})',fontsize=16,fontweight='bold',pad=20)
am.set_ylabel('Price ($)',fontsize=12);am.legend(fontsize=10,loc='upper left');am.grid(True,alpha=0.3);am.tick_params(labelbottom=False)
am.annotate(f'${cp.iloc[-1]:.2f}',xy=(li,cp.iloc[-1]),xytext=(10,10),textcoords='offset points',bbox=dict(boxstyle='round,pad=0.3',facecolor='yellow',alpha=0.7),fontsize=10,fontweight='bold')

av=fig.add_subplot(gs[1,:],sharex=am);vd=data['Volume']/1e6;cols=np.where(cp.diff()>0,'#00ff00','#ff0000');cols[0]='#d62728'
av.bar(ix,vd,color=cols,alpha=0.6,width=1);av.set_ylabel('Volume\n(Millions)',fontsize=10);av.grid(True,alpha=0.3);av.tick_params(labelsize=9)

for pos,y,t,yl,col,ex in[(gs[2,0],data['RSI'],'RSI','RSI','#114f7a',lambda a:(a.axhline(70,color='red',linestyle='--',alpha=0.7,label='Overbought (70)'),a.axhline(30,color='green',linestyle='--',alpha=0.7,label='Oversold (30)'),a.set_ylim(0,100),an(a,data['RSI'].iloc[-1],li,'#9467bd'))),(gs[2,1],data['Volatility']*100,'Volatility','Volatility (%)','#114f7a',lambda a:an(a,(data['Volatility']*100).iloc[-1],li)),(gs[2,2],data['Cumulative_Return']*100,'Cumulative Returns','Cumulative Returns (%)','#114f7a',lambda a:an(a,(data['Cumulative_Return']*100).iloc[-1],li))]:
    a=fig.add_subplot(pos);a.plot(ix,y,color=col,linewidth=2,alpha=0.8);st(a,t,yl,False);ex(a)

amd=fig.add_subplot(gs[3,:2]);amd.plot(ix,macd,label='MACD',color='blue',linewidth=1.5);amd.plot(ix,sig,label='Signal',color='red',linewidth=1)
amd.bar(ix,hist,color='gray',alpha=0.5,width=1);amd.axhline(y=0,color='black',linestyle='--',alpha=0.5);st(amd,'MACD','',True,8)

add=fig.add_subplot(gs[3,2]);add.fill_between(ix,dd,0,color='red',alpha=0.3);st(add,'Max Drawdown (%)','Drawdown (%)',False);an(add,dd.iloc[-1],li,'red')

ast=fig.add_subplot(gs[4,0]);ast.plot(ix,kp,label='%K',color='blue');ast.plot(ix,dp,label='%D',color='red')
ast.axhline(80,color='red',linestyle='--',alpha=0.7,label='Overbought');ast.axhline(20,color='green',linestyle='--',alpha=0.7,label='Oversold');st(ast,'Stochastic Oscillator','')

ah=fig.add_subplot(gs[4,1]);ah.hist(dr.dropna()*100,bins=50,alpha=0.7,color='purple',edgecolor='black')
ah.set_title('Daily Returns Distribution',fontweight='bold',fontsize=11);ah.set_xlabel('Return (%)');ah.axvline(x=0,color='red',linestyle='--');ah.tick_params(axis='both',labelsize=8)

ao=fig.add_subplot(gs[4,2]);ao.plot(ix,obv/1e6,color='orange',linewidth=2);st(ao,'On-Balance Volume (Millions)','',False);ao.grid(True,alpha=0.3)

asr=fig.add_subplot(gs[5,:]);asr.plot(ix,rsh,color='green',linewidth=2);asr.set_title('Rolling Sharpe Ratio (60-day)',fontweight='bold',fontsize=10)
asr.axhline(y=0,color='black',linestyle='--',alpha=0.5);asr.grid(True,alpha=0.3);asr.tick_params(axis='both',labelsize=8)

setup_date_formatting(asr,period);plt.tight_layout();plt.subplots_adjust(bottom=0.15);plt.show()

In [ ]:
# Display key metrics with visual cards and mini charts
from functions import display_stock_metrics

if 'selected_stock' not in globals() or not selected_stock:
    from IPython.display import HTML, display
    display(HTML('<div style="color: #dc2626; font-weight: 600;">No stock selected.</div>'))
else:
    t = yf.Ticker(selected_stock)
    rh = t.history(period='5d')
    
    if 'data' not in globals() or data is None or (isinstance(data, pd.DataFrame) and data.empty):
        data = t.history(period='2y')
        if isinstance(data, pd.DataFrame) and not data.empty:
            from functions import calculate_technical_indicators
            data = calculate_technical_indicators(data)
    
    i = getattr(t, 'info', {}) or {}
    
    # Display the metrics dashboard
    display_stock_metrics(selected_stock, data, t, rh, i)

[Back to Top](#table-of-contents)

## CAPM Analysis (Capital Asset Pricing Model)

CAPM helps determine the expected return of an asset based on its systematic risk (beta) relative to the market.

In [ ]:
# CAPM (Capital Asset Pricing Model) Analysis
from functions import calculate_capm_metrics, display_capm_analysis, plot_capm_analysis

print(f"Calculating CAPM for {selected_stock}...")

# Calculate CAPM metrics
capm_metrics = calculate_capm_metrics(selected_stock, weeks_to_fetch=150, capm_period='4y')

# Extract variables for subsequent analysis
aligned_data = capm_metrics['aligned_data']
beta = capm_metrics['beta']
r_squared = capm_metrics['r_squared']

# Perform linear regression to get slope and intercept for later use
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(aligned_data['Market'], aligned_data['Stock'])

# Display analysis
display_capm_analysis(selected_stock, capm_metrics)

# Create visualizations
plot_capm_analysis(selected_stock, capm_metrics)

print(f"\nVariables extracted for further analysis:")
print(f"  - aligned_data: {len(aligned_data)} weekly returns")
print(f"  - beta: {beta:.3f}")
print(f"  - r_squared: {r_squared:.3f}")

In [ ]:
# Nonlinear Beta Analysis
# Note: Run the CAPM Analysis cell above first to generate required data

# Check if required variables exist
required_vars = ['aligned_data', 'beta', 'r_squared', 'slope', 'intercept']
missing_vars = [v for v in required_vars if v not in globals()]

if missing_vars:
    print(f"ERROR: Missing required variables: {', '.join(missing_vars)}")
    print(f"Please run the 'CAPM Analysis' cell above first to generate these variables.")
    raise SystemExit(f"Missing variables: {', '.join(missing_vars)}")

print(f"\n{'='*60}")
print(f"NONLINEAR BETA ANALYSIS FOR {selected_stock}")
print(f"{'='*60}\n")

# 1. CONDITIONAL BETA: Separate betas for up and down markets
up_market = aligned_data[aligned_data['Market'] > 0]
down_market = aligned_data[aligned_data['Market'] <= 0]

# Calculate beta for up markets
if len(up_market) > 10:
    slope_up, intercept_up, r_up, p_up, std_err_up = stats.linregress(up_market['Market'], up_market['Stock'])
    beta_up = slope_up
else:
    beta_up = np.nan
    
# Calculate beta for down markets
if len(down_market) > 10:
    slope_down, intercept_down, r_down, p_down, std_err_down = stats.linregress(down_market['Market'], down_market['Stock'])
    beta_down = slope_down
else:
    beta_down = np.nan

print("1. CONDITIONAL BETA (Market Direction)")
print(f"   Up Market periods (n={len(up_market)}):    β⁺ = {beta_up:.3f}")
print(f"   Down Market periods (n={len(down_market)}):  β⁻ = {beta_down:.3f}")

if not np.isnan(beta_up) and not np.isnan(beta_down):
    beta_asymmetry = beta_down - beta_up
    print(f"   Asymmetry (β⁻ - β⁺):                     {beta_asymmetry:.3f}")
    
    if abs(beta_asymmetry) > 0.2:
        if beta_down > beta_up:
            print(f"   → Stock is MORE sensitive to market declines (defensive less effective)")
        else:
            print(f"   → Stock is LESS sensitive to market declines (better downside protection)")
    else:
        print(f"   → Relatively symmetric response to market movements")

# 2. QUADRATIC BETA: Polynomial regression to capture curvature
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Prepare data for polynomial regression
X = aligned_data['Market'].values.reshape(-1, 1)
y = aligned_data['Stock'].values

# Fit quadratic model
poly_features = PolynomialFeatures(degree=2, include_bias=True)
X_poly = poly_features.fit_transform(X)

poly_model = LinearRegression()
poly_model.fit(X_poly, y)
y_pred_poly = poly_model.predict(X_poly)

# Extract coefficients: y = β₀ + β₁*x + β₂*x²
beta_0_poly = poly_model.intercept_
beta_1_poly = poly_model.coef_[1]  # Linear term
beta_2_poly = poly_model.coef_[2]  # Quadratic term

r2_poly = r2_score(y, y_pred_poly)
r2_linear = r_squared

print(f"\n2. QUADRATIC BETA (Polynomial Regression)")
print(f"   Model: Return = {beta_0_poly:.4f} + {beta_1_poly:.3f}·Market + {beta_2_poly:.3f}·Market²")
print(f"   Linear coefficient (β₁):    {beta_1_poly:.3f}")
print(f"   Quadratic coefficient (β₂):  {beta_2_poly:.4f}")
print(f"   R² (quadratic model):        {r2_poly:.3f}")
print(f"   R² (linear model):           {r2_linear:.3f}")
print(f"   R² improvement:              {(r2_poly - r2_linear):.4f}")

if abs(beta_2_poly) > 0.5:
    if beta_2_poly > 0:
        print(f"   → Convex relationship: Returns accelerate with larger market moves (both directions)")
    else:
        print(f"   → Concave relationship: Returns decelerate with larger market moves (dampening effect)")
else:
    print(f"   → Weak quadratic effect: Linear model is adequate")

# 3. DYNAMIC BETA: Rolling beta to show time variation
window = 26  # 26 weeks = ~6 months
if len(aligned_data) >= window + 20:
    rolling_beta = []
    rolling_dates = []
    
    for i in range(window, len(aligned_data)):
        window_data = aligned_data.iloc[i-window:i]
        slope_i, _, _, _, _ = stats.linregress(window_data['Market'], window_data['Stock'])
        rolling_beta.append(slope_i)
        rolling_dates.append(aligned_data.index[i])
    
    rolling_beta = np.array(rolling_beta)
    beta_volatility = rolling_beta.std()
    beta_range = rolling_beta.max() - rolling_beta.min()
    
    print(f"\n3. DYNAMIC BETA (26-week Rolling Window)")
    print(f"   Mean rolling beta:           {rolling_beta.mean():.3f}")
    print(f"   Beta volatility (std):       {beta_volatility:.3f}")
    print(f"   Beta range [min, max]:       [{rolling_beta.min():.3f}, {rolling_beta.max():.3f}]")
    
    if beta_volatility > 0.3:
        print(f"   → High beta instability: Risk profile changes significantly over time")
    else:
        print(f"   → Stable beta: Consistent risk profile")
else:
    rolling_beta = None
    print(f"\n3. DYNAMIC BETA: Insufficient data (need {window + 20}+ weeks)")

# Visualization: Nonlinear Beta Analysis
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Conditional Beta (Up vs Down Markets)
ax1 = fig.add_subplot(gs[0, 0])
if not np.isnan(beta_up):
    ax1.scatter(up_market['Market']*100, up_market['Stock']*100, alpha=0.6, s=30, 
                color='green', label=f'Up Market (β⁺={beta_up:.3f})')
    # Fit line for up market
    x_up = up_market['Market'].values
    y_up_fit = intercept_up + slope_up * x_up
    ax1.plot(x_up*100, y_up_fit*100, 'g--', linewidth=2, alpha=0.8)

if not np.isnan(beta_down):
    ax1.scatter(down_market['Market']*100, down_market['Stock']*100, alpha=0.6, s=30, 
                color='red', label=f'Down Market (β⁻={beta_down:.3f})')
    # Fit line for down market
    x_down = down_market['Market'].values
    y_down_fit = intercept_down + slope_down * x_down
    ax1.plot(x_down*100, y_down_fit*100, 'r--', linewidth=2, alpha=0.8)

ax1.axhline(y=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)
ax1.axvline(x=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)
ax1.set_xlabel('Market Return (%)', fontsize=11)
ax1.set_ylabel(f'{selected_stock} Return (%)', fontsize=11)
ax1.set_title('Conditional Beta: Up vs Down Markets', fontsize=12, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: Quadratic Fit
ax2 = fig.add_subplot(gs[0, 1])
ax2.scatter(aligned_data['Market']*100, aligned_data['Stock']*100, alpha=0.4, s=20, color='blue', label='Actual')

# Plot linear fit
x_line = np.linspace(aligned_data['Market'].min(), aligned_data['Market'].max(), 100)
y_linear = intercept + slope * x_line
ax2.plot(x_line*100, y_linear*100, 'r--', linewidth=2, label=f'Linear (β={beta:.3f})', alpha=0.8)

# Plot quadratic fit
x_quad = np.linspace(aligned_data['Market'].min(), aligned_data['Market'].max(), 100).reshape(-1, 1)
X_quad_plot = poly_features.transform(x_quad)
y_quad = poly_model.predict(X_quad_plot)
ax2.plot(x_quad*100, y_quad*100, 'g-', linewidth=2.5, label=f'Quadratic (R²={r2_poly:.3f})', alpha=0.9)

ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)
ax2.axvline(x=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)
ax2.set_xlabel('Market Return (%)', fontsize=11)
ax2.set_ylabel(f'{selected_stock} Return (%)', fontsize=11)
ax2.set_title('Linear vs Quadratic Beta Model', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Dynamic Beta Over Time
ax3 = fig.add_subplot(gs[1, :])
if rolling_beta is not None:
    ax3.plot(rolling_dates, rolling_beta, linewidth=2, color='purple', alpha=0.8)
    ax3.axhline(y=beta, color='red', linestyle='--', linewidth=2, label=f'Overall Beta = {beta:.3f}', alpha=0.7)
    ax3.axhline(y=1.0, color='gray', linestyle='-', linewidth=1, alpha=0.5, label='Market Beta = 1.0')
    ax3.fill_between(rolling_dates, rolling_beta.mean() - beta_volatility, 
                     rolling_beta.mean() + beta_volatility, alpha=0.2, color='purple',
                     label=f'±1 Std Dev ({beta_volatility:.3f})')
    ax3.set_xlabel('Date', fontsize=11)
    ax3.set_ylabel('Rolling Beta (26 weeks)', fontsize=11)
    ax3.set_title(f'Dynamic Beta Evolution - {selected_stock}', fontsize=12, fontweight='bold')
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3)
    ax3.tick_params(axis='x', rotation=45)
else:
    ax3.text(0.5, 0.5, 'Insufficient data for rolling beta analysis', 
             ha='center', va='center', fontsize=12, transform=ax3.transAxes)
    ax3.set_xticks([])
    ax3.set_yticks([])

# Plot 4: Beta Comparison Summary
ax4 = fig.add_subplot(gs[2, 0])
beta_labels = ['Linear\nBeta', 'Up Market\nBeta⁺', 'Down Market\nBeta⁻', 'Quadratic\nβ₁']
beta_values = [beta, beta_up if not np.isnan(beta_up) else 0, 
               beta_down if not np.isnan(beta_down) else 0, beta_1_poly]
colors_bar = ['blue', 'green', 'red', 'purple']

bars = ax4.bar(beta_labels, beta_values, color=colors_bar, alpha=0.7, edgecolor='black', linewidth=1.5)
ax4.axhline(y=1.0, color='gray', linestyle='--', linewidth=1.5, alpha=0.7, label='Market Beta = 1.0')
ax4.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
ax4.set_ylabel('Beta Value', fontsize=11)
ax4.set_title('Beta Comparison Across Methods', fontsize=12, fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}', ha='center', va='bottom' if height > 0 else 'top', 
            fontsize=10, fontweight='bold')

# Plot 5: Residual Analysis
ax5 = fig.add_subplot(gs[2, 1])
residuals_linear = y - (intercept + slope * aligned_data['Market'].values)
residuals_poly = y - y_pred_poly

ax5.scatter(aligned_data['Market']*100, residuals_linear*100, alpha=0.5, s=20, 
           color='red', label='Linear Residuals')
ax5.scatter(aligned_data['Market']*100, residuals_poly*100, alpha=0.5, s=20, 
           color='green', label='Quadratic Residuals')
ax5.axhline(y=0, color='black', linestyle='-', linewidth=1.5, alpha=0.7)
ax5.set_xlabel('Market Return (%)', fontsize=11)
ax5.set_ylabel('Residual (%)', fontsize=11)
ax5.set_title('Residual Analysis: Linear vs Quadratic', fontsize=12, fontweight='bold')
ax5.legend(fontsize=9)
ax5.grid(True, alpha=0.3)

plt.suptitle(f'Nonlinear Beta Analysis - {selected_stock}', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print(f"\n{'='*60}")
print("SUMMARY: Choose the appropriate beta based on:")
print("• Use conditional beta if asymmetry > 0.2 (different up/down behavior)")
print("• Use quadratic beta if R² improvement > 0.05 (nonlinear relationship)")
print("• Monitor dynamic beta for time-varying risk assessment")
print(f"{'='*60}\n")

## Company infomations

In [ ]:
# Render latest news articles
try:
    news = getattr(ticker, 'news', None)
    if not news:
        display(Markdown("**No news available.**"))
    else:
        from datetime import datetime
        md = [f"### News for {selected_stock} — {len(news)} articles found", '']
        
        for i, article in enumerate(news, 1):
            try:
                content = article.get('content', {}) if isinstance(article, dict) else {}
                provider = (content.get('provider', {}) if isinstance(content, dict) else {}).get('displayName', 'Unknown')
                summary = content.get('summary') or content.get('description') or content.get('title') or 'No summary'
                pub_date = content.get('pubDate') or content.get('displayTime') or article.get('pubDate') or article.get('date') or 'Unknown date'
                
                if pub_date and pub_date != 'Unknown date':
                    try:
                        pub_date = datetime.fromisoformat(str(pub_date).replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
                    except:
                        pub_date = str(pub_date)
                
                summary_short = (summary[:150] + '...') if isinstance(summary, str) and len(summary) > 150 else summary
                md.append(f"{i}. **{provider}** — {summary_short} ({pub_date})")
            except Exception as e:
                md.append(f"{i}. Error: {e}")
        
        display(Markdown('\n\n'.join(md)))
except Exception as e:
    display(Markdown(f"**Error:** {e}"))

In [ ]:
# Company Officers, Institutional Holdings, and Mutual Fund Holdings
from functions import display_company_holdings

# Get ticker object and display holdings
ticker_obj = globals().get('ticker') or yf.Ticker(selected_stock)
display_company_holdings(selected_stock, ticker_obj)

In [ ]:
# Fetch and display competitors from CNN Business API
from functions import make_cnn_api_request

try:
    resp = make_cnn_api_request(f"https://production.dataviz.cnn.io/quote/competitors/{selected_stock}/10")
    
    if resp is None:
        raise ValueError("No response received (network error)")
    if resp.status_code != 200:
        raise ValueError(f"Status {resp.status_code}: {resp.text[:1000]}")
    
    data = resp.json()
    items = data.get("competitors", data) if isinstance(data, dict) else data if isinstance(data, list) else []
    
    norm = lambda s: str(s or "").upper().strip().split("-")[0].split(".")[0] if s else ""
    target = norm(selected_stock)
    
    competitors_list = [
        {"symbol": (sym := norm(c.get("symbol") or c.get("ticker") or c.get("ticker_symbol") or "")) or "N/A",
         "name": c.get("name") or c.get("companyName") or "Unknown"}
        for c in items if norm(c.get("symbol") or c.get("ticker") or c.get("ticker_symbol") or "") != target
    ][:10]
    
    if competitors_list:
        md = f"### Top {len(competitors_list)} competitors for {selected_stock}\n\n" + \
             "\n".join(f"{i}. **{c['symbol']}** — {c['name']}" for i, c in enumerate(competitors_list, 1))
    else:
        md = f"**No competitors found**\n\n{json.dumps(items, indent=2)[:4000]}"
    
    display(Markdown(md))
except Exception as e:
    display(Markdown(f"**Error fetching competitors:** {e}"))

### Insider information

In [ ]:
# Insider Transactions
try:
    ins = yf.Ticker(selected_stock).insider_transactions
    if ins is None or ins.empty:
        raise ValueError("No data")
    
    df_ins = ins.copy()
    date_col = next((c for c in ['Start Date', 'FilingDate', 'Date', 'TransactionDate'] if c in df_ins.columns), None)
    if date_col:
        df_ins.sort_values(by=date_col, ascending=False, inplace=True)
    
    lines = [f"### Insider Transactions for {selected_stock}\n"]
    for _, row in df_ins.head(10).iterrows():
        date_str = f"{pd.to_datetime(row.get(date_col, '')).date()}" if pd.notna(row.get(date_col)) else 'Unknown date'
        title = row.get('Title', row.get('Position', '')).strip()
        who = f"**{title}**, {row.get('Insider', 'Unknown')}" if title else row.get('Insider', 'Unknown')
        action = row.get('Text', 'Transaction').strip()
        shares_h = humanize_number(row.get('Shares', row.get('SharesTransacted', row.get('SharesChanged', None))))
        value_h = humanize_number(row.get('Value', row.get('ValueTransacted', None)))
        lines.append(f"- {date_str}: {who} **{action}** ({shares_h} shares) for an overall value of **${value_h}**")
    
    display(Markdown("\n".join(lines)))
except Exception as e:
    print(f'No insider transactions available for {selected_stock}')

In [ ]:
# Format insider_transactions with humanize_number
insider_trans = ticker.insider_transactions.head(10)
numeric_cols = insider_trans.select_dtypes(include=[np.number]).columns
insider_trans[numeric_cols] = insider_trans[numeric_cols].applymap(humanize_number)
insider_trans

In [ ]:
roster_holder = ticker.insider_roster_holders

# Select only numeric columns and apply humanize_number to them
numeric_cols = roster_holder.select_dtypes(include=[np.number]).columns
roster_holder[numeric_cols] = roster_holder[numeric_cols].applymap(humanize_number)

# Display the DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(roster_holder)

[Back to Top](#table-of-contents)

## Financial Statements

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create output widget
output = widgets.Output()

# Create buttons
quarterly_btn = widgets.Button(description='Show Quarterly', button_style='info')
yearly_btn = widgets.Button(description='Show Yearly', button_style='info')
both_btn = widgets.Button(description='Show Both', button_style='success')

# Custom button colors and font
quarterly_btn.style.button_color = '#2a5fff'
quarterly_btn.style.font_weight = 'bold'

yearly_btn.style.button_color = '#fb950f'
yearly_btn.style.font_weight = 'bold'

both_btn.style.button_color = '#00cc66'
both_btn.style.font_weight = 'bold'

def plot_data(show='both'):
    with output:
        clear_output(wait=True)
        
        if show == 'both':
            fig, axes = plt.subplots(2, 2, figsize=(18, 10))
            rows = [(0, ticker.quarterly_income_stmt, ticker.quarterly_balance_sheet, 'Quarterly', 
                     lambda d: [f"Q{((x.month-1)//3)+1} {x.year}" for x in d]),
                    (1, ticker.income_stmt, ticker.balance_sheet, 'Annual', lambda d: [x.year for x in d])]
        elif show == 'quarterly':
            fig, axes = plt.subplots(1, 2, figsize=(18, 5))
            rows = [(0, ticker.quarterly_income_stmt, ticker.quarterly_balance_sheet, 'Quarterly', 
                     lambda d: [f"Q{((x.month-1)//3)+1} {x.year}" for x in d])]
            axes = axes.reshape(1, -1)
        else:  # yearly
            fig, axes = plt.subplots(1, 2, figsize=(18, 5))
            rows = [(0, ticker.income_stmt, ticker.balance_sheet, 'Annual', lambda d: [x.year for x in d])]
            axes = axes.reshape(1, -1)
        
        for row_idx, inc, bal, period, fmt in rows:
            for col, (metrics, title_mid, ratio_label, ylim) in enumerate([
                ({'Revenue': inc.loc["Total Revenue"]/1e9, 'Net Income': inc.loc["Net Income"]/1e9},
                 'Total Revenue and Net Income', 'Profit Margin (%)', None),
                ({'Total Assets': bal.loc["Total Assets"]/1e9, 'Total Debt': bal.loc["Total Debt"]/1e9},
                 'Total Assets and Total Debt', 'Debt-to-Asset Ratio (%)', 100)
            ]):
                df = pd.DataFrame(metrics).sort_index().tail(4)
                ax = axes[row_idx, col]
                df.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=ax)
                ax.set_title(f'{period} {title_mid} (in Billions) with {ratio_label}')
                ax.set_ylabel('Amount (Billions USD)')
                ax.set_xticks(range(len(df)))
                ax.set_xticklabels(fmt(df.index), rotation=0)
                ax.grid(axis='y', linestyle='--', alpha=0.7)
                
                ratio = (df.iloc[:, 1] / df.iloc[:, 0]) * 100
                ax_twin = ax.twinx()
                ax_twin.plot(range(len(ratio)), ratio.values, color='#fb950f', marker='o', linewidth=2, label=ratio_label)
                ax_twin.set_ylabel(ratio_label, color="#000000")
                ax_twin.tick_params(axis='y', labelcolor="#000000")
                ax_twin.set_ylim(bottom=0, top=ylim)
                
                lines1, labels1 = ax.get_legend_handles_labels()
                lines2, labels2 = ax_twin.get_legend_handles_labels()
                ax.legend(lines1 + lines2, labels1 + labels2, title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')
        
        plt.tight_layout()
        plt.show()

# Button click handlers
def on_quarterly_clicked(b):
    plot_data('quarterly')

def on_yearly_clicked(b):
    plot_data('yearly')

def on_both_clicked(b):
    plot_data('both')

quarterly_btn.on_click(on_quarterly_clicked)
yearly_btn.on_click(on_yearly_clicked)
both_btn.on_click(on_both_clicked)

# Display buttons and output (suppress widget repr output)
_ = display(widgets.HBox([quarterly_btn, yearly_btn, both_btn]))
_ = display(output)

# Show yearly by default
plot_data('yearly')

In [ ]:
# Display annual balance sheet with all rows visible
bs = ticker.balance_sheet
bs = bs.applymap(humanize_number)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(bs)

In [ ]:
# Display Balance Sheet as HTML table
display_balance_sheet_html(ticker)

## Analyst Coverage

In [ ]:
# Take a closer look at this - Analyst upgrades and downgrades data
ticker.upgrades_downgrades.head(10)

### Analyst Price Targets Visualization

This chart combines the stock price history with analyst price target distributions from upgrades/downgrades data.

In [ ]:
# Plot Recommendations Summary
plot_analyst_recommendations(ticker, selected_stock)

[Back to Top](#table-of-contents)

## Market Intelligence

In [ ]:
# Fear and Greed Index Data
headers_list = [
    {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", "Accept": "application/json, text/javascript, */*; q=0.01", "Referer": "https://www.cnn.com/"},
    {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36", "Accept": "application/json, text/javascript, */*; q=0.01", "Referer": "https://www.cnn.com/"},
    {"User-Agent": "curl/7.88.1", "Accept": "*/*"}
]

session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])))

resp = None
for h in headers_list:
    try:
        resp = session.get("https://production.dataviz.cnn.io/index/fearandgreed/graphdata", headers=h, timeout=8)
        if resp.status_code == 200:
            break
    except requests.exceptions.RequestException:
        resp = None

if not resp:
    print("No response received")
else:
    def parse_ts(ts):
        if ts is None:
            return pd.NaT
        try:
            if isinstance(ts, (int, float)):
                t = float(ts)
                return pd.to_datetime(t, unit='ms' if t > 1e12 else 's', utc=True) if t > 1e9 else pd.NaT
            if isinstance(ts, str):
                return pd.to_datetime(ts)
        except:
            return pd.NaT
        return pd.NaT

    data = resp.json()
    rows = []
    for mk in ['fear_and_greed', 'fear_and_greed_historical', 'market_momentum_sp500', 'market_momentum_sp125', 'stock_price_strength', 'stock_price_breadth', 'put_call_options', 'market_volatility_vix', 'market_volatility_vix_50', 'junk_bond_demand', 'safe_haven_demand']:
        entry = data.get(mk) if isinstance(data, dict) else None
        score, rating, timestamp = None, None, None
        if isinstance(entry, dict):
            score, rating, timestamp = entry.get('score'), entry.get('rating'), entry.get('timestamp')
            if (score is None or rating is None or timestamp is None) and isinstance(entry.get('data'), list) and entry.get('data'):
                last = entry['data'][-1]
                if isinstance(last, dict):
                    score = score if score is not None else (last.get('y') or last.get('value'))
                    rating = rating if rating is not None else (last.get('rating') or last.get('label'))
                    timestamp = timestamp if timestamp is not None else last.get('x')
        rows.append({'metric': mk, 'score': score, 'rating': rating, 'timestamp': parse_ts(timestamp)})

    df = pd.DataFrame(rows).set_index('metric')
    df['score'] = pd.to_numeric(df['score'], errors='coerce')
    display(df)

In [ ]:
# Fear and greed index - historical trend plot
url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://www.cnn.com/",
}

session = requests.Session()
retries = Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

try:
    resp = session.get(url, headers=headers, timeout=8)
    resp.raise_for_status()
    data = resp.json()
except Exception as e:
    raise SystemExit(f"Error fetching data: {e}")

# --- Parse data ---
def parse_timestamp(ts):
    """Convert CNN timestamp to pandas datetime"""
    return pd.to_datetime(ts, unit='ms', utc=True, errors='coerce')

historical = data.get("fear_and_greed_historical", {}).get("data", [])
if not historical:
    raise SystemExit("No historical data available.")

df = pd.DataFrame([
    {"timestamp": parse_timestamp(item.get("x")), "score": item.get("y")}
    for item in historical if item.get("x") and item.get("y") is not None
])
df = df.dropna().sort_values("timestamp")
df = df[(df["score"] >= 0) & (df["score"] <= 100)]

# --- Plot ---
def get_color(value):
    if value <= 25: return '#DC2626'   # Extreme Fear
    if value <= 45: return '#F97316'   # Fear
    if value <= 55: return '#FCD34D'   # Neutral
    if value <= 75: return '#10B981'   # Greed
    return '#059669'                   # Extreme Greed

def get_rating(value):
    if value <= 25: return 'Extreme Fear'
    if value <= 45: return 'Fear'
    if value <= 55: return 'Neutral'
    if value <= 75: return 'Greed'
    return 'Extreme Greed'

fig, ax = plt.subplots(figsize=(18, 6))

# Main line
ax.plot(df['timestamp'], df['score'], color='#1e3a8a', linewidth=3, alpha=0.9, zorder=3)

# Color fill
for i in range(len(df) - 1):
    x = [df['timestamp'].iloc[i], df['timestamp'].iloc[i + 1]]
    y = [df['score'].iloc[i], df['score'].iloc[i + 1]]
    ax.fill_between(x, 0, y, color=get_color(np.mean(y)), alpha=0.3)

# Reference lines
for y, color in [(25, '#DC2626'), (45, '#F97316'), (55, '#FCD34D'), (75, '#10B981')]:
    ax.axhline(y=y, color=color, linestyle='--', alpha=0.5, linewidth=1.5)

# Latest value annotation
last = df.iloc[-1]
ax.scatter(last['timestamp'], last['score'], color=get_color(last['score']),
           s=200, edgecolors='white', linewidth=3, zorder=5)
ax.annotate(f"{last['score']:.1f}\n{get_rating(last['score'])}",
            xy=(last['timestamp'], last['score']), xytext=(20, 20),
            textcoords='offset points', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.7', facecolor=get_color(last['score']),
                      edgecolor='white', alpha=0.9), color='white')

# Labels & style
ax.set_title("CNN Fear & Greed Index - Historical Trend", fontsize=16, fontweight='bold')
ax.set_xlabel("Date", fontsize=11)
ax.set_ylabel("Score (0–100)", fontsize=11)
ax.set_ylim(0, 100)
ax.grid(alpha=0.25, linestyle=':')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Fetch share-price insights
resp = make_cnn_api_request(f"https://production.dataviz.cnn.io/insights/share_price/{selected_stock}")

def extract_plain_text(obj):
    if not isinstance(obj, (dict, list)):
        return None
    if isinstance(obj, dict):
        for key in ['list_summary', 'listSummary', 'listsummary']:
            if key in obj:
                val = obj[key]
                if isinstance(val, dict):
                    for pk in ('plain_text', 'plainText', 'plaintext'):
                        if pk in val and isinstance(val[pk], str):
                            return val[pk].strip()
                if isinstance(val, str):
                    return val.strip()
        for v in obj.values():
            if res := extract_plain_text(v):
                return res
    if isinstance(obj, list):
        for item in obj:
            if res := extract_plain_text(item):
                return res
    return None

try:
    if not resp:
        display(Markdown("**No response received — cannot fetch insights.**"))
    elif resp.status_code != 200:
        display(Markdown(f"**Status:** {resp.status_code}. Response:\n\n{resp.text[:1000]}"))
    else:
        try:
            data = resp.json()
            if plain := extract_plain_text(data):
                display(Markdown(plain))
            else:
                display(Markdown(f"**No list_summary.plain_text found — JSON (truncated):**\n\n{json.dumps(data, indent=2)[:4000]}"))
        except ValueError:
            display(Markdown(f"**Failed to decode JSON**\n\n{resp.text[:2000]}"))
except Exception as e:
    display(Markdown(f"**Error:** {e}"))

In [ ]:
# Real-time stock price streaming via WebSocket (BTC-USD example)
#!pip install yfinance nest_asyncio

# Import modules
import nest_asyncio
import asyncio

# Allow nested async loops (needed for Jupyter)
nest_asyncio.apply()

# Define a message handler
def message_handler(message):
    # Each message is a dict with price updates
    if "id" in message and message["id"] == "BTC-USD":
        price = message.get("price")
        if price:
            print(f"BTC-USD: ${price:.2f}")

# Create async function for WebSocket 
async def stream_btc():
    async with yf.AsyncWebSocket(verbose=False) as ws:
        await ws.subscribe(["BTC-USD"])
        print("Connected to Yahoo Finance WebSocket. Streaming BTC-USD live...")
        await ws.listen(message_handler)

# Run it
#await stream_btc()         # Uncomment this line to start streaming (will run indefinitely)

In [ ]:
# Define major stock indices with full country names
country_indices = {
    'United States of America': '^GSPC', 'United Kingdom': '^FTSE', 'Germany': '^GDAXI', 'France': '^FCHI',
    'Japan': '^N225', 'China': '000001.SS', 'India': '^BSESN', 'Brazil': '^BVSP', 'Canada': '^GSPTSE',
    'Australia': '^AXJO', 'South Korea': '^KS11', 'Mexico': '^MXX', 'South Africa': '^JN0U.JO',
    'Spain': '^IBEX', 'Italy': 'FTSEMIB.MI', 'Netherlands': '^AEX', 'Switzerland': '^SSMI',
    'Sweden': '^OMX', 'Hong Kong': '^HSI', 'Singapore': '^STI', 'Taiwan': '^TWII', 'Indonesia': '^JKSE',
    'Malaysia': '^KLSE', 'Thailand': '^SET.BK', 'Norway': '^OSEAX', 'Denmark': '^OMXC25',
    'Finland': '^OMXH25', 'Belgium': '^BFX', 'Austria': '^ATX', 'Turkey': 'XU100.IS',
    'Poland': 'WIG20.WA', 'Israel': '^TA125.TA', 'Argentina': '^MERV', 'Chile': '^IPSA', 'New Zealand': '^NZ50'
}

warnings.filterwarnings('ignore')

# Import world map functions
from functions import get_daily_changes, create_world_map, save_world_map

print("Fetching stock market data...")
df = get_daily_changes(country_indices)
print(f"Successfully fetched data for {len(df)} countries")

print("Creating world map...")
world_map = create_world_map(df)
map_path = save_world_map(world_map, output_dir='assets')
print(f"Map saved to: {map_path}")

# Display the map in notebook
from IPython.display import display, HTML
display(world_map)

## Options

[Back to Top](#table-of-contents)

In [ ]:
expirations = ticker.options
print(expirations)

expiry = expirations[0]  # Take the nearest expiry
option_chain = ticker.option_chain(expiry)

calls = option_chain.calls
puts = option_chain.puts

print(calls.head())
print(puts.head())

In [ ]:
put_call_ratio = puts['openInterest'].sum() / calls['openInterest'].sum()
print(f"Put/Call Open Interest Ratio: {put_call_ratio:.2f}")

In [ ]:
max_call_strike = calls.loc[calls['openInterest'].idxmax(), 'strike']
max_put_strike = puts.loc[puts['openInterest'].idxmax(), 'strike']
print(f"Max Call OI Strike: {max_call_strike}")
print(f"Max Put OI Strike: {max_put_strike}")

In [ ]:
atm_strike = calls.iloc[(calls['strike'] - ticker.history(period="1d")['Close'].iloc[-1]).abs().argsort()[:1]]['strike'].values[0]
atm_call = calls[calls['strike'] == atm_strike]['lastPrice'].values[0]
atm_put = puts[puts['strike'] == atm_strike]['lastPrice'].values[0]
expected_move = atm_call + atm_put
print(f"Approx. Expected Move by {expiry}: ±${expected_move:.2f}")

In [ ]:
iv_data = []
for exp in expirations[:5]:  # Check first few expirations
    oc = ticker.option_chain(exp)
    iv = oc.calls['impliedVolatility'].mean()
    iv_data.append({'expiry': exp, 'avg_IV': iv})

iv_df = pd.DataFrame(iv_data)
print(iv_df)

In [ ]:
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts
spot = ticker.history(period="1d")['Close'].iloc[-1]

# Calculate median and mean strikes
call_median_strike = calls['strike'].median()
put_median_strike = puts['strike'].median()
call_mean_strike = (calls['strike'] * calls['openInterest']).sum() / calls['openInterest'].sum()
put_mean_strike = (puts['strike'] * puts['openInterest']).sum() / puts['openInterest'].sum()

# Create subplots
fig, axs = plt.subplots(3, 1, figsize=(12, 12))

# First subplot: Open Interest by Strike
axs[0].bar(calls['strike'], calls['openInterest'], width=1.0, label='Calls OI', color='green', alpha=0.6)
axs[0].bar(puts['strike'], puts['openInterest'], width=1.0, label='Puts OI', color='red', alpha=0.6)
axs[0].axvline(spot, color='blue', linestyle='--', label=f'Spot Price (${spot:.2f})', linewidth=2)
axs[0].axvline(call_median_strike, color='darkgreen', linestyle=':', label=f'Call Median (${call_median_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].axvline(put_median_strike, color='darkred', linestyle=':', label=f'Put Median (${put_median_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].axvline(call_mean_strike, color='lime', linestyle='-.', label=f'Call Mean (${call_mean_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].axvline(put_mean_strike, color='orange', linestyle='-.', label=f'Put Mean (${put_mean_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].set_title(f'Open Interest by Strike — {expiry}')
axs[0].set_xlabel('Strike Price')
axs[0].set_ylabel('Open Interest')
axs[0].legend()

# Second subplot: Put/Call Open Interest Ratio by Expiry
ratios = []
for exp in ticker.options[:6]:
    oc = ticker.option_chain(exp)
    pcr = oc.puts['openInterest'].sum() / oc.calls['openInterest'].sum()
    ratios.append({'expiry': exp, 'put_call_ratio': pcr})

df_ratios = pd.DataFrame(ratios)
axs[1].plot(df_ratios['expiry'], df_ratios['put_call_ratio'], marker='o')
axs[1].set_title('Put/Call Open Interest Ratio by Expiry')
axs[1].set_xlabel('Expiration Date')
axs[1].set_ylabel('Put/Call Ratio')
axs[1].grid(True)

# Third subplot: Implied Volatility Term Structure
term = []
for exp in ticker.options[:8]:
    oc = ticker.option_chain(exp)
    iv_mean = (oc.calls['impliedVolatility'].mean() + oc.puts['impliedVolatility'].mean()) / 2
    term.append({'expiry': exp, 'avg_IV': iv_mean})

df_term = pd.DataFrame(term)
axs[2].plot(df_term['expiry'], df_term['avg_IV'], marker='o', color='purple')
axs[2].set_title('Implied Volatility Term Structure')
axs[2].set_xlabel('Expiration Date')
axs[2].set_ylabel('Average Implied Volatility')
axs[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# --- Load data ---
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts
spot = ticker.history(period="1d")['Close'].iloc[-1]

# --- Prepare data ---
strikes = np.array(sorted(set(calls['strike']).intersection(puts['strike'])))

# Select 10 strikes around the spot price
closest_idx = np.argmin(abs(strikes - spot))
start_idx = max(0, closest_idx - 4)
end_idx = min(len(strikes), closest_idx + 5)
strikes = strikes[start_idx:end_idx]

call_oi = calls.set_index('strike').reindex(strikes)['openInterest'].fillna(0)
put_oi  = puts.set_index('strike').reindex(strikes)['openInterest'].fillna(0)

# --- Plot settings ---
bar_width = 0.4
x = np.arange(len(strikes))

plt.figure(figsize=(12,6))
plt.bar(x - bar_width/2, call_oi, width=bar_width, color='green', alpha=0.7, label='Calls OI')
plt.bar(x + bar_width/2, put_oi,  width=bar_width, color='red', alpha=0.7, label='Puts OI')

# --- Add current stock price marker ---
closest_strike = strikes[np.argmin(abs(strikes - spot))]
plt.axvline(x[np.where(strikes == closest_strike)[0][0]], color='blue', linestyle='--', label=f'Spot Price (${spot:.2f})')

# --- Labels and style ---
plt.title(f"Open Interest by Strike — {expiry}")
plt.xlabel("Strike Price")
plt.ylabel("Open Interest")
plt.xticks(x, strikes, rotation=45)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# --- Load option chain ---
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts

# --- Merge call and put open interest by strike ---
oi_table = pd.merge(
    calls[['strike', 'openInterest', 'volume', 'impliedVolatility']],
    puts[['strike', 'openInterest', 'volume', 'impliedVolatility']],
    on='strike',
    suffixes=('_call', '_put')
)

# --- Sort by strike ---
oi_table = oi_table.sort_values(by='strike').reset_index(drop=True)

# --- Format the columns for readability ---
oi_table.columns = [
    "Strike",
    "Call OI",
    "Call Volume",
    "Call IV",
    "Put OI",
    "Put Volume",
    "Put IV"
]

# --- Round and clean up ---
oi_table["Call IV"] = (oi_table["Call IV"] * 100).round(2)
oi_table["Put IV"] = (oi_table["Put IV"] * 100).round(2)

print(f"Open Interest Table — {expiry}")
display(oi_table.head(20))  # show first 20 rows